In [1]:
import h2o

In [2]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) 64-Bit Server VM (build 25.211-b12, mixed mode)
  Starting server from C:\Users\Raghav\anaconda3\Lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\Raghav\AppData\Local\Temp\tmp6dvljyrl
  JVM stdout: C:\Users\Raghav\AppData\Local\Temp\tmp6dvljyrl\h2o_Raghav_started_from_python.out
  JVM stderr: C:\Users\Raghav\AppData\Local\Temp\tmp6dvljyrl\h2o_Raghav_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,09 secs
H2O_cluster_timezone:,Europe/London
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.32.1.3
H2O_cluster_version_age:,3 months and 3 days
H2O_cluster_name:,H2O_from_python_Raghav_fp928b
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,1.754 Gb
H2O_cluster_total_cores:,0
H2O_cluster_allowed_cores:,0
H2O_cluster_status:,"accepting new members, healthy"


In [4]:
df = h2o.import_file(r"C:\Users\Raghav\Desktop\Solar Flare\Data\Final Data\sunspots_cleaned.csv")

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [5]:
df

magnetic_classification,zurich_class,penumbra_class,compactness_class,No_of_spots,longitudanal_extent,Area,Status
A,H,R,X,1,1,30,Unassociated
B,C,R,O,2,3,10,Unassociated
B,C,R,O,5,10,20,Unassociated
B,C,A,O,2,10,30,Unassociated
B,E,S,O,5,12,120,Unassociated
A,H,S,X,1,2,60,Unassociated
B,C,R,O,4,4,60,Unassociated
A,H,K,X,1,5,180,Unassociated
B,D,S,O,2,10,100,Unassociated
B,C,R,O,4,10,60,Unassociated


In [6]:
df.shape

(7568, 8)

In [7]:
df.describe()

Rows:7568
Cols:8




,magnetic_classification,zurich_class,penumbra_class,compactness_class,No_of_spots,longitudanal_extent,Area,Status
type,enum,enum,enum,enum,int,int,int,enum
mins,,,,,1.0,0.0,0.0,
mean,,,,,2.7400898520084565,3.9689482029598313,63.480972515856216,
maxs,,,,,64.0,24.0,2400.0,
sigma,,,,,3.215894777120409,3.1848413349836067,100.33059221187526,
zeros,,,,,0,303,293,
missing,0,0,0,0,0,0,0,0
0,A,H,R,X,1.0,1.0,30.0,Unassociated
1,B,C,R,O,2.0,3.0,10.0,Unassociated
2,B,C,R,O,5.0,10.0,20.0,Unassociated


In [8]:
df.columns
df['magnetic_classification'] = df['magnetic_classification'].asfactor()
df['zurich_class'] = df['zurich_class'].asfactor()
df['penumbra_class'] = df['penumbra_class'].asfactor()
df['compactness_class'] = df['compactness_class'].asfactor()


In [9]:
df

magnetic_classification,zurich_class,penumbra_class,compactness_class,No_of_spots,longitudanal_extent,Area,Status
A,H,R,X,1,1,30,Unassociated
B,C,R,O,2,3,10,Unassociated
B,C,R,O,5,10,20,Unassociated
B,C,A,O,2,10,30,Unassociated
B,E,S,O,5,12,120,Unassociated
A,H,S,X,1,2,60,Unassociated
B,C,R,O,4,4,60,Unassociated
A,H,K,X,1,5,180,Unassociated
B,D,S,O,2,10,100,Unassociated
B,C,R,O,4,10,60,Unassociated


In [10]:
response_column = 'Status'
df[response_column] = df[response_column].asfactor()

In [11]:
predictors = df.columns
del predictors[-1]
predictors

['magnetic_classification',
 'zurich_class',
 'penumbra_class',
 'compactness_class',
 'No_of_spots',
 'longitudanal_extent',
 'Area']

In [12]:
#Split the data for Machine Learning

train, test = df.split_frame(
    ratios=[0.8,], 
    seed=101, 
    destination_frames=['train.hex','test.hex']
)


In [13]:
print(train.shape)
# print(valid.shape)
print(test.shape)

(6071, 8)
(1497, 8)


# Forest

In [14]:
forestGrid = h2o.grid.H2OGridSearch(
    h2o.estimators.H2ORandomForestEstimator(
        nfolds = 10),
    hyper_params = {
        'ntrees' : [50, 100, 200, 500],
        'max_depth' : [4, 6, 7, 8, 10]
    })

# Define model

forestGrid.train(x = predictors, y= response_column, training_frame = train)


drf Grid Build progress: |████████████████████████████████████████████████| 100%


In [15]:
forestGrid

     max_depth ntrees  \
0           10     50   
1           10    500   
2           10    200   
3           10    100   
4            8    500   
5            8    200   
6            8    100   
7            8     50   
8            7    100   
9            7    500   
10           7    200   
11           7     50   
12           6    100   
13           6     50   
14           6    200   
15           6    500   
16           4     50   
17           4    200   
18           4    500   
19           4    100   

                                                   model_ids  \
0    Grid_DRF_train.hex_model_python_1629662051381_1_model_5   
1   Grid_DRF_train.hex_model_python_1629662051381_1_model_20   
2   Grid_DRF_train.hex_model_python_1629662051381_1_model_15   
3   Grid_DRF_train.hex_model_python_1629662051381_1_model_10   
4   Grid_DRF_train.hex_model_python_1629662051381_1_model_19   
5   Grid_DRF_train.hex_model_python_1629662051381_1_model_14   
6    Grid_DRF_train.hex_mo

In [16]:
from h2o.estimators import H2ORandomForestEstimator
forest = H2ORandomForestEstimator(ntrees = 50, max_depth = 7, nfolds = 10)

#Train
forest.train(x=predictors, y = response_column, training_frame = train)

drf Model Build progress: |███████████████████████████████████████████████| 100%


In [17]:
forest

Model Details
H2ORandomForestEstimator :  Distributed Random Forest
Model Key:  DRF_model_python_1629662051381_29066


Model Summary: 


,,number_of_trees,number_of_internal_trees,model_size_in_bytes,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
0,,50.0,50.0,46362.0,7.0,7.0,7.0,55.0,87.0,69.12




ModelMetricsBinomial: drf
** Reported on train data. **

MSE: 0.18900721485826003
RMSE: 0.43474960018182884
LogLoss: 0.5597473411979413
Mean Per-Class Error: 0.2880702255897606
AUC: 0.7883827307448205
AUCPR: 0.7634935553393327
Gini: 0.5767654614896409

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.35329544736959073: 


,,Associated,Unassociated,Error,Rate
0,Associated,1479.0,1553.0,0.5122,(1553.0/3032.0)
1,Unassociated,306.0,2733.0,0.1007,(306.0/3039.0)
2,Total,1785.0,4286.0,0.3062,(1859.0/6071.0)



Maximum Metrics: Maximum metrics at their respective thresholds


...

,,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid,cv_6_valid,cv_7_valid,cv_8_valid,cv_9_valid,cv_10_valid
0,accuracy,0.7085232,0.02192444,0.7116041,0.73333335,0.71428573,0.74437296,0.71843004,0.7035775,0.6739865,0.6810631,0.6914378,0.713141
1,auc,0.78853655,0.019621326,0.7655259,0.80807996,0.7887631,0.82989186,0.7938693,0.78005713,0.7725467,0.76893824,0.78167295,0.7960201
2,err,0.2914768,0.02192444,0.2883959,0.26666668,0.2857143,0.255627,0.28156996,0.29642248,0.3260135,0.31893688,0.3085622,0.28685898
3,err_count,176.8,12.035133,169.0,168.0,178.0,159.0,165.0,174.0,193.0,192.0,191.0,179.0
4,f0point5,0.69178945,0.02215885,0.707127,0.71584105,0.7039801,0.72282606,0.70509523,0.68489444,0.66152406,0.6622058,0.67209774,0.6823028
5,f1,0.75220203,0.011791564,0.75036925,0.7579251,0.7607527,0.7698987,0.7685835,0.7441176,0.74571806,0.74934727,0.73435324,0.7409551
6,f2,0.8252041,0.021889081,0.7992448,0.8052664,0.8274854,0.8235294,0.84463626,0.8145525,0.8544686,0.86289835,0.80931944,0.8106396
7,lift_top_group,1.7522528,0.22542495,1.2850877,1.8210863,1.4950593,1.9418082,1.670267,2.017182,1.843386,1.7257525,1.7553312,1.9675676
8,logloss,0.5611544,0.01547467,0.57999194,0.54313624,0.5625345,0.52949715,0.5560134,0.5662098,0.57270485,0.57704234,0.566179,0.55823535
9,max_per_class_error,0.46699563,0.07968205,0.4219858,0.37223974,0.46179402,0.37261146,0.48056537,0.45945945,0.6040956,0.5940594,0.47936508,0.4237805



Scoring History: 


,,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_auc,training_pr_auc,training_lift,training_classification_error
0,,2021-08-22 21:01:12,8.102 sec,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1,,2021-08-22 21:01:12,8.107 sec,1.0,0.449671,0.715090,0.754052,0.735111,1.778170,0.328278
2,,2021-08-22 21:01:12,8.113 sec,2.0,0.448840,0.734294,0.755954,0.745348,1.785820,0.332786
3,,2021-08-22 21:01:12,8.120 sec,3.0,0.444748,0.676309,0.764043,0.754594,1.836592,0.322347
4,,2021-08-22 21:01:12,8.129 sec,4.0,0.446004,0.680210,0.760326,0.747474,1.833951,0.316508
5,,2021-08-22 21:01:12,8.140 sec,5.0,0.444705,0.654638,0.763762,0.745250,1.744467,0.323820
6,,2021-08-22 21:01:12,8.154 sec,6.0,0.442992,0.636877,0.767791,0.749226,1.833951,0.322637
7,,2021-08-22 21:01:12,8.168 sec,7.0,0.441911,0.622895,0.770418,0.751035,1.772150,0.307534
8,,2021-08-22 21:01:12,8.185 sec,8.0,0.441084,0.599362,0.772108,0.751057,1.768453,0.305228
9,,2021-08-22 21:01:12,8.202 sec,9.0,0.439624,0.580266,0.775410,0.751725,1.768453,0.314204



See the whole table with table.as_data_frame()

Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,Area,4642.606934,1.000000,0.337394
1,zurich_class,3309.481934,0.712850,0.240511
2,penumbra_class,2486.593994,0.535603,0.180709
3,longitudanal_extent,1621.702515,0.349309,0.117855
4,No_of_spots,1170.048340,0.252024,0.085031
5,compactness_class,284.054382,0.061184,0.020643
6,magnetic_classification,245.693604,0.052921,0.017855


In [18]:
performance_forest = forest.model_performance(test_data=test)

print(performance_forest)


ModelMetricsBinomial: drf
** Reported on test data. **

MSE: 0.18671716541714778
RMSE: 0.4321078168896598
LogLoss: 0.5541926933007943
Mean Per-Class Error: 0.2753882264743681
AUC: 0.7952716692845924
AUCPR: 0.7736168849766527
Gini: 0.5905433385691847

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.3514924079179764: 


,,Associated,Unassociated,Error,Rate
0,Associated,379.0,373.0,0.496,(373.0/752.0)
1,Unassociated,67.0,678.0,0.0899,(67.0/745.0)
2,Total,446.0,1051.0,0.2939,(440.0/1497.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.351492,0.755011,253.0
1,max f2,0.301090,0.866009,293.0
2,max f0point5,0.540174,0.725735,133.0
3,max accuracy,0.419186,0.724115,213.0
4,max precision,0.860539,1.000000,0.0
5,max recall,0.135592,1.000000,385.0
6,max specificity,0.860539,1.000000,0.0
7,max absolute_mcc,0.410010,0.459507,219.0
8,max min_per_class_accuracy,0.486786,0.714094,169.0
9,max mean_per_class_accuracy,0.410010,0.724612,219.0



Gains/Lift Table: Avg response rate: 49.77 %, avg score: 49.92 %


,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain,kolmogorov_smirnov
0,1,0.026720,0.836653,1.908926,1.908926,0.950000,0.841022,0.950000,0.841022,0.051007,0.051007,90.892617,90.892617,0.048347
1,2,0.044756,0.824223,1.786130,1.859441,0.888889,0.824570,0.925373,0.834392,0.032215,0.083221,78.612975,85.944105,0.076573
2,3,0.052104,0.813659,2.009396,1.880589,1.000000,0.818514,0.935897,0.832153,0.014765,0.097987,100.939597,88.058854,0.091338
3,4,0.128257,0.790835,1.551113,1.684962,0.771930,0.795748,0.838542,0.810537,0.118121,0.216107,55.111268,68.496225,0.174884
4,5,0.150301,0.782779,1.461379,1.652170,0.727273,0.785947,0.822222,0.806931,0.032215,0.248322,46.137889,65.217002,0.195131
5,6,0.205745,0.717620,1.791510,1.689719,0.891566,0.740788,0.840909,0.789107,0.099329,0.347651,79.150966,68.971934,0.282491
6,7,0.302605,0.668591,1.205638,1.534770,0.600000,0.695088,0.763797,0.759012,0.116779,0.464430,20.563758,53.477043,0.322142
7,8,0.400802,0.564401,1.339597,1.486953,0.666667,0.616303,0.740000,0.724049,0.131544,0.595973,33.959732,48.695302,0.388526
8,9,0.502338,0.483554,1.176554,1.424213,0.585526,0.527071,0.708777,0.684234,0.119463,0.715436,17.655422,42.421284,0.424213
9,10,0.599866,0.416127,1.114802,1.373908,0.554795,0.451036,0.683742,0.646320,0.108725,0.824161,11.480188,37.390771,0.446501


In [19]:
import numpy as np
print('Precision: ', forest.precision())
print('recall: ',  forest.recall())
print('accuracy: ',  np.average(forest.accuracy(test, performance_forest)))
print('sensitivity: ', forest.sensitivity())
print('specificity: ', forest.specificity())
print('F1: ', forest.F1())



Precision:  [[0.9314465761184693, 1.0]]
recall:  [[0.07332231244470322, 1.0]]
accuracy:  0.555085193602362
sensitivity:  [[0.07332231244470322, 1.0]]
specificity:  [[0.9314465761184693, 1.0]]
F1:  [[0.35329544736959073, 0.7462116040955632]]


In [20]:
pred = forest.predict(test)

NameError: name 'pd' is not defined

In [ ]:
pred

In [ ]:
#Feature Importance
exm = forest.explain(test)

# Gradient Boosted Trees

In [ ]:
gbmGrid = h2o.grid.H2OGridSearch(
    h2o.estimators.H2OGradientBoostingEstimator(
        nfolds = 10),
    hyper_params = {
        'ntrees' : [50, 100, 200, 500],
        'max_depth' : [4, 2, 6, 7, 8, 10],
        'learn_rate' : [0.01, 0.05, 0.06]
    })

# Define model

gbmGrid.train(x = predictors, y= response_column, training_frame = train)


In [ ]:
gbmGrid

In [ ]:
#GBT
from h2o.estimators.gbm import H2OGradientBoostingEstimator
from h2o.grid.grid_search import H2OGridSearch
gbm = H2OGradientBoostingEstimator(ntrees = 120 , max_depth = 7 )
gbm.train(x=predictors, y=response_column, training_frame=train)

## Show a detailed model summary
print(gbm)

In [ ]:
## Get the AUC on the validation set
perf = gbm.model_performance(test)
perf
# print(perf.auc())

In [ ]:
pred = gbm.predict(test)

In [ ]:
print('Precision: ', gbm.precision())
print('recall: ', gbm.recall())
print('accuracy: ', gbm.accuracy())
print('sensitivity: ', gbm.sensitivity())
print('specificity: ', gbm.specificity())
print('F1: ', gbm.F1())



In [ ]:
#Feature Importance
exm = gbm.explain(test)

# Neural Nets

In [ ]:
from h2o.estimators.deeplearning import H2ODeepLearningEstimator

In [ ]:
deep = h2o.grid.H2OGridSearch(
    h2o.estimators.H2ODeepLearningEstimator(
        nfolds = 10),
    hyper_params = {
        'epochs' : [10, 50, 100, 200],
        'hidden' : [[10, 10], [5, 5]],
        'activation' : ['Tanh', 'Rectifier']
    })

# Define model

deep.train(x = predictors, y= response_column, training_frame = train)

In [ ]:
model = H2ODeepLearningEstimator(
        distribution="multinomial",
        hidden=[200,200,200],
        epochs=100,
        nfolds = 10)
model.train(x=predictors, y=response_column, training_frame=train)

In [ ]:
model

In [ ]:
## Get the AUC on the validation set
perf = model.model_performance(test)
perf
# print(perf.auc())

In [ ]:
print('Precision: ', model.precision())
print('recall: ', model.recall())
print('accuracy: ', model.accuracy())
print('sensitivity: ', model.sensitivity())
print('specificity: ', model.specificity())
print('F1: ', model.F1())



In [ ]:
#Feature Importance
exm = model.explain(test)

# Logistic Regression

In [ ]:
from h2o.estimators.glm import H2OGeneralizedLinearEstimator

In [ ]:
regression = H2OGeneralizedLinearEstimator(nfolds = 10, family = 'binomial' )
regression.train(x=predictors, y=response_column, training_frame=train)
print(regression)

In [ ]:
regression_perf = regression.model_performance(test)
regression_perf

In [ ]:
#Feature Importance
exm = regression.explain(test)

# SVM

In [ ]:
from h2o.estimators import H2OSupportVectorMachineEstimator

In [ ]:
svm_model = H2OSupportVectorMachineEstimator(gamma=0.01,
                                             rank_ratio = 0.1,
                                             disable_training_metrics = False)
svm_model.train(x=predictors, y=response_column, training_frame=train)
svm_model

In [ ]:
svm_perf = svm_model.model_performance(test)
svm_perf

In [ ]:
#Feature Importance
exm = svm_model.explain(test)